Please install `pip install PyNiteFEA==0.0.63` before running the test

In [5]:
from PyNite import FEModel3D

In [6]:
from PyNite import FEModel3D
from PyNite.Visualization import render_model
from math import sqrt
from math import tan
from math import radians

In [15]:
def frame_cal(H:float, L:float, w:float, ang_d:float):
    """
    H, l, w, ang_d
    """
    
    frame_model = FEModel3D() # Creates an empty model
    H  # mm
    L  # mm
    w  # N/mm
    ang = radians (ang_d) # rad

    #Add nodes
    frame_model.add_node("node1",0,0,0)
    frame_model.add_node("node2",0,H,0)
    frame_model.add_node("node3",L/2,H+((L/2)*tan(ang)),0)
    frame_model.add_node("node4",L,H,0)
    frame_model.add_node("node5",L,0,0)

    m = (L/2)**2 + ((L/2)*tan(ang))**2
    member_length = sqrt(m)

    #Define material
    I_1 = 9050000.0 # mm*4
    I_2 = 672000.0 # mm*4
    J = 60500.0 # mm*4
    A = 2300.0 # mm*2
    E = 200e3
    nu = 0.25
    rho = 7.85e-6
    G = E/(2 * (1+2*nu))
    
    frame_model.add_material('Steel', E, G, nu, rho)

    #Add frame members
    frame_model.add_member('M1', 'node1', 'node2', 'Steel', I_2, I_1, J, A)
    frame_model.add_member('M2', 'node2', 'node3', 'Steel', I_2, I_1, J, A)
    frame_model.add_member('M3', 'node3', 'node4', 'Steel', I_2, I_1, J, A)
    frame_model.add_member('M4', 'node4', 'node5', 'Steel', I_2, I_1, J, A)
    
    #Add load
    frame_model.add_load_combo('L',{"D":1})
    frame_model.add_member_dist_load("M1", "Fy", -w, -w, 0, H, case="D")
    frame_model.add_member_dist_load("M2", "Fy", w, w, 0, member_length, case="D")
    frame_model.add_member_dist_load("M3", "Fy", w, w, 0, member_length, case="D")
    frame_model.add_member_dist_load("M4", "Fy", w, w, 0, H, case="D")

    #Add boundary condition
    frame_model.def_support('node1', True, True, True, True, True, False)
    frame_model.def_support('node5', True, True, True, True, True, False)

    
    #render_model(frame_model, combo_name="L", annotation_size=0.1)
    
    #Submit for analysis
    frame_model.analyze()

    #Node Vertical Displacement
    node_v_1 = frame_model.Nodes['node1'].DY
    node_v_2 = frame_model.Nodes['node2'].DY
    node_v_3 = frame_model.Nodes['node3'].DY
    node_v_4 = frame_model.Nodes['node4'].DY
    node_v_5 = frame_model.Nodes['node5'].DY
    node_displ_v = max(node_v_1['L'], node_v_2['L'], node_v_3['L'], node_v_4['L'], node_v_5['L'])

    #Node Horizontal Displacement
    node_h_1 = frame_model.Nodes['node1'].DX
    node_h_2 = frame_model.Nodes['node2'].DX
    node_h_3 = frame_model.Nodes['node3'].DX
    node_h_4 = frame_model.Nodes['node4'].DX
    node_h_5 = frame_model.Nodes['node5'].DX
    node_displ_h = max(node_h_1['L'], node_h_2['L'], node_h_3['L'], node_h_4['L'], node_h_5['L'])
    
    #max momemnt
    moment_max_1=abs(frame_model.Members['M1'].max_moment('Mz', combo_name = 'L')/10**6)
    moment_max_2=abs(frame_model.Members['M2'].max_moment('Mz', combo_name = 'L')/10**6)
    moment_max_3=abs(frame_model.Members['M3'].max_moment('Mz', combo_name = 'L')/10**6)
    moment_max_4=abs(frame_model.Members['M4'].max_moment('Mz', combo_name = 'L')/10**6)
    #min moment
    moment_min_1=abs(frame_model.Members['M1'].min_moment('Mz', combo_name = 'L')/10**6)
    moment_min_2=abs(frame_model.Members['M2'].min_moment('Mz', combo_name = 'L')/10**6)
    moment_min_3=abs(frame_model.Members['M3'].min_moment('Mz', combo_name = 'L')/10**6)
    moment_min_4=abs(frame_model.Members['M4'].min_moment('Mz', combo_name = 'L')/10**6)
    moment = max (moment_max_1, moment_max_2, moment_max_3, moment_max_4, moment_min_1, moment_min_2, moment_min_3, moment_min_4)

    #max shear
    shear_max_1=abs(frame_model.Members['M1'].max_shear('Fy', combo_name = 'L')/10**3)
    shear_max_2=abs(frame_model.Members['M2'].max_shear('Fy', combo_name = 'L')/10**3)
    shear_max_3=abs(frame_model.Members['M3'].max_shear('Fy', combo_name = 'L')/10**3)
    shear_max_4=abs(frame_model.Members['M4'].max_shear('Fy', combo_name = 'L')/10**3)
    
    #min shear
    shear_min_1=abs(frame_model.Members['M1'].min_shear('Fy', combo_name = 'L')/10**3)
    shear_min_2=abs(frame_model.Members['M2'].min_shear('Fy', combo_name = 'L')/10**3)
    shear_min_3=abs(frame_model.Members['M3'].min_shear('Fy', combo_name = 'L')/10**3)
    shear_min_4=abs(frame_model.Members['M4'].min_shear('Fy', combo_name = 'L')/10**3)
    shear = max(shear_max_1, shear_max_2, shear_max_3, shear_max_4, shear_min_1, shear_min_2, shear_min_3, shear_min_4)
    
    return moment, shear, node_displ_v, node_displ_h

In [36]:
def Strand7(H:float, ang_d:float, w:float, L:float = 0.0001, delat:float= 0.5):
    H = H * 1000.0
    L= L * 1000.0
    
    D_h = 0
    L_D_h = L
    while D_h <= H/150:
        D_h = frame_cal(H, L_D_h, w, ang_d)[3]
        print (L_D_h, D_h)
        L_D_h+=delat

    
    L_D_h = L_D_h-delat * 2
    D_h = frame_cal(H, L_D_h, w, ang_d)[3]
    print ('D_h criteria: ', L_D_h/1000, D_h)

In [37]:
Strand7(3,10,2.1)

0.1 0.0
0.6 1023052.8042858598
D_h criteria:  0.00010000000000000009 0.0


In [12]:
def Strand7(H:float, ang_d:float, w:float, L:float = 1.0, delat:float= 0.1):
    H = H * 1000.0
    L= L * 1000.0
    
    M = 0.0
    L_M = L
    while M <= 35.03:
        M = frame_cal(H, L_M, w, ang_d)[0]
        L_M+=delat
    
    L_M = L_M-delat * 2
    M = frame_cal(H, L_M, w, ang_d)[0]
    print ('Moment criteria: ', L_M/1000, M)


    V = 0
    L_V = L + 35000
    while V <= 146.88:
        V = frame_cal(H, L_V, w, ang_d)[1]
        L_V+=delat
    L_V = L_V-delat * 2
    V = frame_cal(H, L_V, w, ang_d)[1]
    print ('Shear criteria: ', L_V/1000, V)

    
    D_h = 0
    L_D_h = L
    while D_h <= H/150:
        D_h = frame_cal(H, L_D_h, w, ang_d)[3]
        L_D_h+=delat
    L_D_h = L_D_h-delat * 2
    D_h = frame_cal(H, L_D_h, w, ang_d)[3]
    print ('D_h criteria: ', L_D_h/1000, D_h)

    
    D_v = 0
    L_D_v= L
    while D_v <= L_D_v/250:
        D_v = frame_cal(H, L_D_v, w, ang_d)[2]
        L_D_v+=delat
    L_D_v = L_D_v-delat * 2
    D_v = frame_cal(H, L_D_v, w, ang_d)[2]    
    
    print ('D_v criteria: ', L_D_v/1000, D_v)
    
    L_old = min (L_M, L_V, L_D_h, L_D_v)

    M_old = frame_cal(H, L_old, w, ang_d)[0]
    print ('Old Moment Max', M_old)
    print('Old L max', L_old )
    
    L_new = 2 * L_old
    M_new = frame_cal(H, L_new, w, ang_d)[0]
    print ('New Moment Max', M_new)   
    print('New L max', L_new)
    #return L_M, L_V, L_D_h, L_D_v
    

In [13]:
Strand7(3,10,2.1)

Moment criteria:  13.786800000032466 35.029931867421645
Shear criteria:  69.26839999978746 146.87975268070085
D_h criteria:  0.9999 22.9981348484372


KeyboardInterrupt: 

In [9]:
Strand7(3,15,2.1)

Moment criteria:  14.409 35.02800091066197
Shear criteria:  84.502 146.87883455622446
D_h criteria:  0.999 23.066975117550097
D_v criteria:  10.975 43.900675611521365
Old Moment Max 9.507962192760242
Old L max 999.0
New Moment Max 9.794852892159494
New L max 1998.0


In [7]:
Strand7(3.1,10,2)

Moment criteria:  16.246 35.027673902151115
Shear criteria:  56.521 146.87741075475947
D_h criteria:  13.584 20.66455005625099
D_v criteria:  11.7 46.80223220371277
Old Moment Max 20.520353866431417
Old L max 11700.0
New Moment Max 67.76104185470827
New L max 23400.0


In [8]:
Strand7(3.1,17,2.7)

Moment criteria:  14.219 35.02662572731221
Shear criteria:  68.037 146.8790038530801
D_h criteria:  10.997 20.66658862858625
D_v criteria:  12.571 50.28430229464696
Old Moment Max 24.667177549399444
Old L max 10997.0
New Moment Max 65.82757853351667
New L max 21994.0


In [9]:
Strand7(3.2,13,2.2)

Moment criteria:  15.518 35.02972121873805
Shear criteria:  63.785 146.87944190933274
D_h criteria:  12.319 21.331140582440113
D_v criteria:  11.973 47.893741849239326
Old Moment Max 23.617521181184465
Old L max 11973.0
New Moment Max 71.514247814501
New L max 23946.0


In [10]:
Strand7(3.2,17,2.4)

Moment criteria:  15.242 35.02754223437663
Shear criteria:  76.56 146.87865169092248
D_h criteria:  11.517 21.332896224798006
D_v criteria:  13.165 52.658135423101314
Old Moment Max 23.806314747601554
Old L max 11517.0
New Moment Max 63.638306898065615
New L max 23034.0


In [11]:
Strand7(3.3,13,2.8)

Moment criteria:  13.151 35.02663359908375
Shear criteria:  51.697 146.87978138477305
D_h criteria:  10.281 21.99787419909893
D_v criteria:  10.438 41.75308338610338
Old Moment Max 25.06425339884939
Old L max 10281.0
New Moment Max 71.57218117399657
New L max 20562.0


In [12]:
Strand7(3.3,14,2.7)

Moment criteria:  13.544 35.02941079432201
Shear criteria:  56.568 146.87733089419152
D_h criteria:  10.457 21.99949576657061
D_v criteria:  10.904 43.616320286634526
Old Moment Max 24.631421089641503
Old L max 10457.0
New Moment Max 69.24025404072529
New L max 20914.0


In [13]:
Strand7(3.3,15,2.2)

Moment criteria:  15.643 35.027184381514125
Shear criteria:  73.034 146.8799659422636
D_h criteria:  11.892 21.99762674174128
D_v criteria:  12.536 50.14698097008716
Old Moment Max 23.59845302614886
Old L max 11892.0
New Moment Max 66.9267743779617
New L max 23784.0


In [14]:
Strand7(3.3,17,2.3)

Moment criteria:  15.511 35.02880510959482
Shear criteria:  79.896 146.8797963829208
D_h criteria:  11.493 21.99726788838092
D_v criteria:  13.168 52.67176126423224
Old Moment Max 23.26599603507647
Old L max 11493.0
New Moment Max 61.93603000778714
New L max 22986.0


In [15]:
Strand7(3.4,10,2.5)

Moment criteria:  13.943 35.02825128852371
Shear criteria:  48.405 146.87610985945904
D_h criteria:  11.171 22.665799796221183
D_v criteria:  10.186 40.74278384103436
Old Moment Max 22.614686592171754
Old L max 10186.0
New Moment Max 66.91941006658611
New L max 20372.0


In [16]:
Strand7(3.5,13,2.4)

Moment criteria:  14.293 35.02918763825212
Shear criteria:  59.551 146.87734301346717
D_h criteria:  10.727 23.330599593821752
D_v criteria:  10.954 43.81816256035965
Old Moment Max 23.692355970731544
Old L max 10727.0
New Moment Max 67.18014105933167
New L max 21454.0


In [17]:
Strand7(3.6,13,2.3)

Moment criteria:  14.564 35.02659872353479
Shear criteria:  62.031 146.87910916904093
D_h criteria:  10.697 23.99836964362495
D_v criteria:  11.04 44.16024992002614
Old Moment Max 23.147908292146806
Old L max 10697.0
New Moment Max 64.70781900367604
New L max 21394.0


In [18]:
Strand7(3.6,18,2.1)

Moment criteria:  16.144 35.02951101528604
Shear criteria:  93.561 146.87981361949437
D_h criteria:  11.102 23.998595434510946
D_v criteria:  13.485 53.93879499346957
Old Moment Max 21.686821024248786
Old L max 11102.0
New Moment Max 54.83392762220325
New L max 22204.0


In [19]:
Strand7(3.6,18,2.5)

Moment criteria:  14.196 35.027392520720554
Shear criteria:  78.321 146.87988856299978
D_h criteria:  9.66 23.997567848572398
D_v criteria:  12.15 48.602522620559526
Old Moment Max 22.21674304362454
Old L max 9660.0
New Moment Max 53.6575536480532
New L max 19320.0


In [20]:
Strand7(3.7,15,2.5)

Moment criteria:  13.711 35.02755908821275
Shear criteria:  65.06 146.87930062284295
D_h criteria:  9.413 24.666170408693773
D_v criteria:  10.961 43.84680215601905
Old Moment Max 22.409006593694393
Old L max 9413.0
New Moment Max 55.794483796793514
New L max 18826.0


In [21]:
Strand7(3.7,18,2.7)

Moment criteria:  13.177 35.02926731462588
Shear criteria:  72.479 146.8798536767644
D_h criteria:  8.545 24.665760845658568
D_v criteria:  11.408 45.62998806077748
Old Moment Max 22.016112343619483
Old L max 8545.0
New Moment Max 49.57134308731417
New L max 17090.0


In [22]:
Strand7(3.8,14,2.3)

Moment criteria:  14.323 35.028876626387394
Shear criteria:  66.275 146.87904400400888
D_h criteria:  9.785 25.33239117834883
D_v criteria:  11.017 44.07006853462166
Old Moment Max 22.03087572650111
Old L max 9785.0
New Moment Max 56.11343663695842
New L max 19570.0


In [23]:
Strand7(3.9,10,2.9)

Moment criteria:  11.821 35.02711719574756
Shear criteria:  45.12 146.87579105996153
D_h criteria:  7.352 25.99853326288895
D_v criteria:  9.122 36.488165309971436
Old Moment Max 22.39493056628265
Old L max 7352.0
New Moment Max 47.0753864238066
New L max 14704.0


In [10]:
Strand7(3.9,12,2)

Moment criteria:  12.605 35.029853677853616
Shear criteria:  83.203 146.87788780800497
D_h criteria:  0.999 60.923793301935206
D_v criteria:  9.918 39.67288847871076
Old Moment Max 15.256208716134257
Old L max 999.0
New Moment Max 15.498558100400404
New L max 1998.0
